Introduction
------------

This notebook aims to introduce a standard way of

  1. loading the data into python notebook
  2. Visual and identify issues
  3. Pre-process the data e.g., variable transformation, normalization and etc.

In [ ]:
## Load packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
from ggplot import *

In [ ]:
## Load data into Python

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

Step 1: Take a peak
-------------------

Now take a peak at what's in the data. This including how many rows and columns and what are they.

In [ ]:
print(train.columns)
print(train.shape)
print(test.shape)

In [ ]:
train.head()

In [ ]:
## Describe the output field
print(train['price_doc'].describe())
sns.distplot(train['price_doc'])

The dependent variable is skewed (as expected for dollars). The easiest would be to take a log tranformation

In [ ]:
train['LogAmt']=np.log(train.price_doc+0.5)
print(train['LogAmt'].describe())
sns.distplot(train['LogAmt'])

** Note the two small bars between 13 and 14, and 14 and 15. We will need to dig into that and see that happens there **.

In [ ]:
## Merge data into one dataset to prepare compare between train and test
train_1 = train.copy()
train_1['Source']='Train'
test_1 = test.copy()
test_1['Source']='Test'
alldata = pd.concat([train_1, test_1])
print(alldata.shape)

Most fields are numeric, and a few of them are object. 

In [ ]:
## Numerical and Categorical data types
alldata_dtype=alldata.dtypes
display_nvar = len(alldata.columns)
alldata_dtype_dict = alldata_dtype.to_dict()
alldata.dtypes.value_counts()

Step 2: Data Visualization¶
---------------------------

**Variable Description**

I wrote this function with intension to compare train/test data and check if some variable is illy behaved. It is modified a little to fit this dataset to compared between normal/fraud subset.

It can be applied to both numeric and object data types:

  1. When the data type is object, it will output the value count of each categories
  2. When the data type is numeric, it will output the quantiles
  3. It also seeks any missing values in the dataset

In [ ]:
def var_desc(dt):
    print('--------------------------------------------')
    for c in alldata.columns:
        if alldata[c].dtype==dt:
            t1 = alldata[alldata.Source=='Train'][c]
            t2 = alldata[alldata.Source=='Test'][c]
            if dt=="object":
                f1 = t1[pd.isnull(t1)==False].value_counts()
                f2 = t2[pd.isnull(t2)==False].value_counts()
            else:
                f1 = t1[pd.isnull(t1)==False].describe()
                f2 = t2[pd.isnull(t2)==False].describe()
            m1 = t1.isnull().value_counts()
            m2 = t2.isnull().value_counts()
            f = pd.concat([f1, f2], axis=1)
            m = pd.concat([m1, m2], axis=1)
            f.columns=['Train','Test']
            m.columns=['Train','Test']
            print(dt+' - '+c)
            print('UniqValue - ',len(t1.value_counts()),len(t2.value_counts()))
            print(f.sort_values(by='Train',ascending=False))
            print()

            m_print=m[m.index==True]
            if len(m_print)>0:
                print('missing - '+c)
                print(m_print)
            else:
                print('NO Missing values - '+c)
            if dt!="object":
                if len(t1.value_counts())<=10:
                    c1 = t1.value_counts()
                    c2 = t2.value_counts()
                    c = pd.concat([c1, c2], axis=1)
                    f.columns=['Train','Test']
                    print(c)
            print('--------------------------------------------')

In [ ]:
var_desc('int64')

In [ ]:
var_desc('float64')

In [ ]:
var_desc('object')

Correlation
-----------

Correlation is useful to find peers of input field so we are aware when building models, either to transform them (principal component) or remove one of the two.

The first plot below the overall correlation matrix and there are blocks of variables that are highly correlated

The second plot shows the highly correlated variables with response. 

In [ ]:
## Correlation
corrmat = train.corr()
f, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(corrmat, vmax=.8, square=True,xticklabels=False,yticklabels=False,cbar=False);

In [ ]:
# Top 20 correlated variables
corrmat = train.corr()
k = 20 #number of variables for heatmap
cols = corrmat.nlargest(k, 'price_doc')['price_doc'].index
cm = np.corrcoef(train[cols].values.T)
f, ax = plt.subplots(figsize=(12, 12))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=False, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Detailed Look
-------------

Let's take a detailed look of the variables with response variable. 

In [ ]:
dt = 'object'
sel_col = train.columns[train.dtypes==dt]
sel_col = [t for t in sel_col if t not in ['Id','timestamp','LogAmt']] 

for sc in sel_col:
    data_1  = pd.concat([train[sc], train.LogAmt], axis=1)
    data_2  = pd.melt(data_1,id_vars='LogAmt')
    data_2  = data_2[pd.isnull(data_2.value)==False]
    p = ggplot(data_2, aes(x='value',y='LogAmt')) + geom_boxplot(alpha=0.5,) + theme(plot_margin = dict(right = 1, top=0.5), axis_title_x=sc)
    print(p)

In [ ]:
dt = 'int64'
sel_col = train.columns[train.dtypes==dt]
sel_col = [t for t in sel_col if t not in ['Id','timestamp','LogAmt']] 

for sc in sel_col:
    data_1  = pd.concat([train[sc], train.LogAmt], axis=1)
    data_2  = pd.melt(data_1,id_vars='LogAmt')
    data_2  = data_2[pd.isnull(data_2.value)==False]
    p = ggplot(data_2, aes(x='value',y='LogAmt')) + geom_point(alpha=0.5) + theme(plot_margin = dict(right = 1, top=0.5), axis_title_x=sc)
    print(p)